In [ ]:
!pip install seaborn -U

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from nltk.stem import WordNetLemmatizer
import time

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Here I will be finetuning GPT2 on the askreddit questions data

# Some light EDA

Reddit users are a seemingly curious bunch. The community known as [Ask Reddit](https://www.reddit.com/r/AskReddit/) boasts over 30M members and even has its own [wikipedia entry](https://en.wikipedia.org/wiki/AskReddit), according to which:
> The subreddit describes its focus as "to ask and answer questions that elicit thought-provoking discussions"

Here we will explore a small portion of the questions posted to the AskReddit forum.

## What is reddit asking?

take a sample of questions from the dataset

In [ ]:
df = pd.read_csv("/kaggle/input/askreddit-questions-and-answers/reddit_questions.csv", delimiter=";")
for i in df.sample(n=10)['text']:
    print(i, "\n")

It really seems like reddit is a place to ask almost anything. Can we find some patterns in the data?

In [ ]:
questions = list(df['text'])
questions = [q.replace('""', '"') for q in questions]
counts = {}
for question in questions:
    for word in word_tokenize(question.lower()):
        if word.isalpha():
            try:
                counts[word] += 1
            except KeyError:
                counts[word] = 1
sorted_count = {k: v for k, v in sorted(counts.items(), key=lambda item: item[1])[::-1]}

In [ ]:
px.bar(x=list(sorted_count.keys())[:50], y=list(sorted_count.values())[:50], 
       title="50 most common words", 
       labels={"x":"word", "y":"count"})

Perhaps unsurprisingly, this looks very similar the most common words in the eglish language, except with words like "what", "how", "why" and of course "reddit" appearing more frequently than in common parlance. Does excluding stop words shed more light on what reddit users are asking?

In [ ]:
top_non_stop_words = []
top_non_stop_vals = []
for word in sorted_count.keys():
    if word not in stopwords.words('english'):
        top_non_stop_words.append(word)
        top_non_stop_vals.append(sorted_count[word])

In [ ]:
px.bar(x=top_non_stop_words[:50], y=top_non_stop_vals[:50],
       title="50 most common non stop-words", 
       labels={"x":"word", "y":"count"})

Reddit sure likes talking about reddit.

We've already seen that the word 'what' is the second most common word in the whole dataset. How frequently do users ask "what" questions compared to other questions?

In [ ]:
keys = ["who", "what", "where", "why", "when", "how"]
vals = [counts[k] for k in keys]
vals, keys = zip(*(sorted(zip(vals, keys)))[::-1])

In [ ]:
px.bar(x=list(keys), y=list(vals),
       title="word count",
       labels={"x":"word", "y":"count"})

## question length

In [ ]:
l = [len(q) for q in questions]
sns.displot(l)
plt.vlines(np.median(l), 0, 4500, colors='r')
plt.title("question length")
plt.show()
print("median:", np.median(l))

It takes most people around 79 characters to ask a question, but some questions require a short prose to truly get their point across such as:

In [ ]:
questions[np.argmax(l)]

Reddit really is a place to ask anything.

## Interest in topics over time

(anything measuring term fequency, especially over time, should be taken with a whole heap of salt as the dataset is most likely affected by sampling bias)

In [ ]:
p = sns.histplot(df['timestamp'])

xticklabels = []
for t in p.get_xticks():
    dt = datetime.fromtimestamp(t)
    xticklabels.append(f"{dt.day}/{dt.month}/{dt.year}")
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    p.set_xticklabels(xticklabels, rotation = 45, ha="right")
plt.title("questions posted over time")
plt.show()

It seems that in early 2020, when people around the word were more bored and confused than ever before, there was a massive spike in questions posted to AskReddit. But this may be due to sampling biases.

In [ ]:
def interest_over_time(terms, bins=100):
    ts = []
    for index, row in df.iterrows():
        text = row['text'].lower()
        tokens = word_tokenize(text)
        for t in terms:
            if t in tokens:
                ts.append(row['timestamp'])
    p = sns.histplot(ts, bins=bins)
    
    xticklabels = []
    for t in p.get_xticks():
        dt = datetime.fromtimestamp(t)
        xticklabels.append(f"{dt.day}/{dt.month}/{dt.year}")
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        p.set_xticklabels(xticklabels, rotation = 45, ha="right")
    
    plt.title(f"interest in {'/'.join(terms)} over time")
    
    plt.show()

In [ ]:
interest_over_time(["coronavirus", "covid", "sars-cov-2"])

In [ ]:
for index, row in df.sort_values("timestamp").iterrows():
    t = row["text"].lower()
    if "coronavirus" in t or "covid" in t or "cov-2" in t:
        break
row['text'], row['datetime']

In [ ]:
interest_over_time(["trump"])

In [ ]:
for index, row in df.sort_values("timestamp").iterrows():
    t = row["text"].lower()
    if "trump" in word_tokenize(t):
        break
row['text']

Searching terms such as 'trump' can have misleading results as the word has multiple meanings.

In [ ]:
for index, row in df.sort_values("timestamp").iterrows():
    t = row["text"].lower()
    if "donald trump" in t:
        break
row['text'], row['datetime']

In [ ]:
interest_over_time(["biden"])

In [ ]:
for index, row in df.sort_values("timestamp").iterrows():
    t = row["text"].lower()
    if "biden" in t:
        break
row['text'], row['datetime']

... clearly.

## upvotes

In [ ]:
sns.kdeplot(df['votes'], cut=0)
plt.show()

In [ ]:
df['votes'].describe()

The votes are heavily skewed by just a few posts recieving tens of thousands of votes vs the median (50 percentile) votes of just 6. This is likely due to higher voted posts being shown to more users creating a self-reinforcing feedback.

Here is a sample of questions plotted against number of votes and dates posted to explore.

There is a weakly positive trend in the upper limit of votes with recency, but overall, newer questions seem to be clustering towards the lower end of the votes axis.

In [ ]:
sample = df.sample(n=1000)
px.scatter(sample, x="timestamp", y="votes", hover_name="text", log_y=True, hover_data=['datetime'])

In [ ]:
def to_date(timestamp):
    dt = datetime.fromtimestamp(timestamp)
    return f"{dt.day}/{dt.month}/{dt.year}"

In [ ]:
df['x'] = pd.cut(df['timestamp'], bins=100)
px.bar(x=[to_date(v) for v in df.groupby("x").median()['timestamp'].values],
          y=df.groupby("x").mean()['votes'].values, title="Median upvotes per post over time")

It seems that with the increasing number to questions submitted from mid 2019 to early 2020, there was a corresponding decrease in the average votes per question. Are people just posting boring questions?

# Embeddings

Here I am exploring different sentence embedding & visualisation strategies

* Vectorise each sentence in TF-IDF form
* Dimensionality reduction with LSA for visualisation


* Vectorise each sentence in TF-IDF form
* Dimensionality reduction with LSA and UMAP for visualisation


* Vectorise each sentence with universal sentence encoder
* Dimensionality reduction with PCA for visualisation

In [ ]:
# when running in a kaggle kernel these plots might cause lag
# or crashes when plotting the entire dateset, taking just a sample here
sample = df.sample(frac=0.25)
samples_qs = list(sample['text'])

In [ ]:
q_types = []
w = ['who', 'what', 'where', 'why', 'how', 'when']

for q in samples_qs:
    q = word_tokenize(q.lower())
    f = False
    for i in w:
        if i in q:
            f = True
            q_types.append(i)
            break
    if not f:
        q_types.append("other")

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([q.lower() for q in samples_qs])
svd = TruncatedSVD(n_components=2)
y = svd.fit_transform(X)

px.scatter(x=y[:, 0], y=y[:, 1],
              hover_data=[samples_qs, sample['votes']],
              color=q_types,
              title="latent semantic analysis reduction of TF-IDF, coloured by question type")

What are you in the 1% of? An oddly appropriate embedding for that question.


Exploring some of the data above, you can see that this has captured some of the semantics of the questions being asked, but it doesn't really give much interesting insight. A caveat of TF-IDF embedding seems to be that the length of the text is a significant factor.

UMAP is another dimensionality reduction algorithm used for visualisation.

Here, the sparse TF-IDF representation is reduced to a more manageable 100 dimensions before UMAP to improve the efficiency of UMAP. (this may still be quite slow)

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatised = [lemmatizer.lemmatize(q.lower()) for q in samples_qs]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(lemmatised)
svd = TruncatedSVD(n_components=100)
y = svd.fit_transform(X)

reducer = umap.UMAP()
scaled = StandardScaler().fit_transform(y)
embedding = reducer.fit_transform(scaled)

px.scatter(x=embedding[:, 0], y=embedding[:, 1],
              hover_data=[samples_qs, sample['votes']],
              color=sample['timestamp'],
              title="UMAP of lemmatised TF-IDF, coloured by votes")

Exploring the plot above, there seem to be some clusters of semantically similar questions.

## Universal Sentence Encoder

The universal sentence encoder is a more sophisticated embedding algirithm from Google. The pretrained neural network is optimised for sentence encoding and outputs a single 512 dimension vector per input.

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
pca = PCA()
y = pca.fit_transform(model(samples_qs).numpy())
px.scatter(x=y[:, 0], y=y[:, 1],
              hover_data=[samples_qs, sample['votes']],
              color=sample['timestamp'],
              title="PCA of encoded, coloured by age")

Exploring the plot above it seems that newer posts tend to cluster towards the mean of the projected embedding. Obviously more thorough analysis is needed before any conclusions can be drawn, but it's fun to explore the interactive plot :)

# Asking questions with GPT-2

I will be using the pretrained distilled gpt2 model from 🤗 [Hugging Face](https://huggingface.co/)

In [ ]:
!pip install transformers -U

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [ ]:
#download pretrained model
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token 
model = TFGPT2LMHeadModel.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)

## Pretrained text generation

see https://huggingface.co/blog/how-to-generate for a very nice explanation of text generation methods

In [ ]:
for prompt in ["What is", "Who is", "Why do", "Where are", "How do", "When is"]:

    input_ids = tokenizer.encode(prompt, return_tensors='tf')

    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=50, 
        top_p=0.92, 
        top_k=0,
        num_return_sequences=3
    )

    print(100 * '-' + "\nPrompt:", prompt)
    print("Output:\n" + 100 * '-')
    for i, sample_output in enumerate(sample_outputs):
        print("{}: {}\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

## Fine tuning

In [ ]:
# only train on questions less than 150 chars long
q2 = [q for q in questions if len(q) < 150]
# encode using pretrained tokenizer
train_encodings = tokenizer(q2, truncation=True, padding='max_length', max_length=150)#, return_tensors='tf')

In [ ]:
batch_size = 16
epochs = 2

train_dataset = tf.data.Dataset.from_tensor_slices(
    train_encodings['input_ids']
    ).shuffle(1000)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.experimental.AUTOTUNE)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
steps = len(train_dataset)

for e in range(epochs):
    losses = []
    start_time = time.time()
    for i, inputs in train_dataset.enumerate():
        i = i.numpy()
        with tf.GradientTape() as tape:
            loss_value = model(inputs, labels=inputs)['loss']

        grad = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grad, model.trainable_variables))

        losses.append(tf.reduce_mean(loss_value).numpy())

        total_time = time.time() - start_time
        time_per_step = total_time / (i+1)
        eta = time_per_step * (steps - (i+1))
        m, s = divmod(eta, 60)
        h, m = divmod(m, 60)

        print(f"\repoch {e+1}/{epochs}, step {i+1}/{steps}, loss:{np.mean(losses):.5f}, ETA:{int(h)}h{int(m)}m{int(s)}s", end="", flush=True)
    print()

## fine tuned text generation

In [ ]:
for prompt in ["What is", "Who is", "Why do", "Where are", "How do", "When is", "Why does my", "What is your", "Where do", "If you could"]:

    input_ids = tokenizer.encode(prompt, return_tensors='tf')

    sample_outputs = model.generate(
        input_ids, 
        do_sample=True, 
        max_length=50, 
        top_p=0.92, 
        top_k=0,
        num_return_sequences=3
    )

    print(100 * '-' + "\nPrompt:", prompt)
    print("Output:\n" + 100 * '-')
    for i, sample_output in enumerate(sample_outputs):
        print("{}: {}\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))